In [ ]:
# NECESSARY IMPORTS

import mercury as mr
import ee
import geemap.foliumap as geemap
import os
import datetime
import webbrowser
from colorama import Fore

In [ ]:
# INITIALIZING INTERFACE ELEMENTS

mr.Markdown(text = '**🌎 GAIA - Geospatial & Aerial Images Analyser [BETA] 🌎**')
mr.Markdown(text = '**🌎 This program solution is not final and some details are potentially to be changed 🌎**')
mr.Markdown(text = '**🌏 Current version - 0.3b 🌏**')

text = 'OSINT-TECHNOLOGIES'
text_params = {
    'fontsize': 10,
    'fontcolor': 'black',
    'bold': True,
    'padding': '3px',
    'background': True,
    'bg_color': 'white',
    'border_radius': '1px',
    'position': 'bottomleft',
}

sources_list = ["Google EE"]
control_panel_choice = mr.Select(value="Google EE", choices=sources_list, label="Choose the source control panel")

app  = mr.App(title="GAIA v0.3b [BETA] 🌐",
             description="Open-Source Geospatial & Aerial Images Analyser",
             show_code=False,
             show_prompt=False,
             continuous_update=True,
             static_notebook=False,
             show_sidebar=True,
             full_screen=True,
             allow_download=False)

if control_panel_choice.value == "Google EE":
    ee_user_guide = mr.Button(label="How to start Google EE")
    ee_cred = mr.Button(label="Create EE credentials file")
    ee_reg = mr.Button(label="EE registration")
    save_map_html = mr.Button(label="Save current map [HTML]")
    ds_info = mr.Checkbox(value=False, label='Show datasets technical info')
    date_one = mr.Text(value="2022-01-01", label="Enter 1st date (format YYYY-MM-DD)", rows=1)
    date_two = mr.Text(value="2022-01-02", label="Enter 2nd date (format YYYY-MM-DD)", rows=1)
    datasets_landsat = mr.Checkbox(value=False, label="Show Landsat datasets")

    if datasets_landsat.value:
        ls7bands = mr.MultiSelect(label="Select LandSat 7 SR bands",
                                  value=['SR_B3', 'SR_B2', 'SR_B1'],
                                  choices=['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'SR_ATMOS_OPACITY', 'SR_CLOUD_QA'])

        ls8bands = mr.MultiSelect(label="Select LandSat 8 bands",
                                  value=['B4', 'B3', 'B2'],
                                  choices=['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'QA_PIXEL'])

        ls9bands = mr.MultiSelect(label="Select LandSat 9 SR bands",
                                  value=['SR_B4', 'SR_B3', 'SR_B2'],
                                  choices=['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'SR_QA_AEROSOL'])


        ls9rawbands = mr.MultiSelect(label="Select LandSat 9 RAW bands",
                                  value=['B4', 'B3', 'B2'],
                                  choices=['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'QA_PIXEL'])


    datasets_sentinel = mr.Checkbox(value=False, label="Show Sentinel datasets")

    if datasets_sentinel.value:
        sen1sarbands = mr.MultiSelect(label="Select Sentinel 1 SAR GRD bands",
                                  value=['HH'],
                                  choices=['HH', 'VV', 'VH', 'HV'])

        sen2msibands = mr.MultiSelect(label="Select Sentinel 2 MSI SR bands",
                                  value=['B2', 'B3', 'B4'],
                                  choices=['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B', 'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60'])

    datasets_modis = mr.Checkbox(value=False, label="Show MODIS datasets")
    datasets_other = mr.Checkbox(value=False, label='Show other datasets')

mr.Markdown(text = "#GOOGLE EARTH ENGINE MAP 🌐")
mr.Markdown(text = "**DATASETS TIME PERIOD: from {} to {}**".format(date_one.value, date_two.value))

if ds_info.value:
    try:
        with open("data/ee_ds.txt", "r") as periods:
            mr.Markdown(text = "**AVAILABLE TIME-PERIODS AND BANDS FOR GEE DATASETS**")
            line = periods.readline()
            while line:
                line = periods.readline()
                mr.Markdown(text=line)
    except FileNotFoundError as error:
        print(Fore.RED + '[ERROR. File or directory was not found] -{}'.format(str(error)[9:]))

if ee_user_guide.clicked:
    try:
        with open("data/ee_guide.txt", "r") as guide:
            mr.Markdown(text = "**HOW TO START WORKING WITH GOOGLE EARTH ENGINE**")
            line = guide.readline()
            while line:
                line = guide.readline()
                mr.Markdown(text=line)
    except FileNotFoundError as error:
        print(Fore.RED + '[ERROR. File or directory was not found] -{}'.format(str(error)[9:]))

In [ ]:
# GOOGLE EE AUTHENTICATION AND INITIALIZATION

if ee_reg.clicked:
    webbrowser.open_new_tab('https://code.earthengine.google.com/register')


if ee_cred.clicked:
    exec(open("create_ee_creds.py").read())

filename = 'ee_creds.txt'
if filename not in os.listdir():
    print(Fore.RED + 'No credentials file detected in local directory. Create this file using button in the dashboard')
    mr.Stop()
else:
    creds_txt = open(r'ee_creds.txt', 'r+')
    lines = creds_txt.readlines()
    key_file = "{}".format(lines[0].rstrip())
    service_account = "{}".format(lines[1].rstrip())
    credentials = ee.ServiceAccountCredentials(service_account, key_file)
    ee.Initialize(credentials)

In [ ]:
# MAP AND DATASETS SETTINGS

m = geemap.Map()
m.add_text(text, **text_params)


if datasets_landsat.value:

    #LANDSAT 7
    ls7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2").filterDate(str(date_one.value), str(date_two.value))
    vis_ls7 = {'bands': ls7bands.value}

    #LANDSAT 8
    ls8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA").filterDate(str(date_one.value), str(date_two.value))
    vis_ls8 = {
        'bands': ls8bands.value,
        'min': 0.0,
        'max': 0.4,
    }

    #LANDSAT 9
    def apply_scale_factors(image):
        opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
        thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
        return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

    collection = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').filterDate(str(date_one.value), str(date_two.value))
    median = collection.median()
    ls9 = apply_scale_factors(median)

    vis_natural_ls9 = {
        'bands': ls9bands.value,
        'min': 0.0,
        'max': 0.3,
    }

    #LANDSAT 9 RAW
    ls9raw = ee.ImageCollection("LANDSAT/LC09/C02/T2").filterDate(str(date_one.value), str(date_two.value)) #.filter(ee.Filter.lt('CLOUD_COVER', ccp.value))
    trueColor432 = ls9raw.select(ls9rawbands.value)
    trueColor432Vis = {
        'min': 0.0,
        'max': 30000.0,
    }


if datasets_sentinel.value:

    #SENTINEL 1 SAR GRD
    sen1sg = ee.ImageCollection("COPERNICUS/S1_GRD")
    vis_sen1sar = {'bands': ['HH', 'HV']}


    #SENTINEL 2 MSI
    sen2msi = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate(str(date_one.value), str(date_two.value))
    sen2msi_vis= {
        min: 0.0,
        max: 0.3,
        'bands': ['B2', 'B3', 'B4'],
    }


    #SENTINEL 5P CLOUDS
    sen5pc = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_CLOUD").select('cloud_fraction').filterDate(str(date_one.value), str(date_two.value))
    band_viz = {
        min: 0,
        max: 0.95,
    }


if datasets_modis.value:
    modis_sc = ee.ImageCollection('MODIS/061/MOD10A1').filter(ee.Filter.date('2018-01-01', '2018-05-01'))
    snowCover = modis_sc.select('NDSI_Snow_Cover')
    modissc_vis = {
        min: 0.0,
        max: 100.0,
    }


if datasets_other.value:
    firms = ee.ImageCollection("FIRMS").filterDate(str(date_one.value), str(date_two.value))
    firesVis = {
    min: 325.0,
    max: 400.0,
    }

    ghs = ee.Image("CSP/ERGo/1_0/Global/ALOS_topoDiversity")
    alosTopographicDiversity = ghs.select('constant')
    alosTopographicDiversityVis = {
    min: 0.0,
    max: 1.0,
    }



In [ ]:
# MAPS AND DATASETS INITIALIZATION


m.add_basemap('SATELLITE')
m.add_basemap('TERRAIN')
m.add_basemap('HYBRID')

try:
    if datasets_landsat.value:

        #LANDSAT 7
        m.addLayer(ls7, vis_ls7, "LandSat 7 SR", True, 0.7)

        #LANDSAT8
        m.addLayer(ls8, vis_ls8, 'LandSat 8 TOA')

        #LANDSAT 9
        m.addLayer(ls9, vis_natural_ls9, 'LandSat 9 SR')

        #LANDSAT 9 RAW
        m.addLayer(trueColor432, trueColor432Vis, 'LandSat 9 RAW')



    if datasets_sentinel.value:
        m.addLayer(sen1sg, vis_sen1sar, 'Sentinel 1 SAR GRD')  #SENTINEL 1 SAR GRD
        m.addLayer(sen2msi.median(), sen2msi_vis, 'Sentinel 2 MSI')  #SENTINEL 2 MSI
        m.addLayer(sen5pc, band_viz, 'Sentinel 5P Cloud')  #SENTINTEL 5P CLOUD


    if datasets_modis.value:
        m.addLayer(snowCover, modissc_vis, 'MODIS snowcower')  #MODIS snowcower



    if datasets_other.value:
        m.addLayer(firms, firesVis, 'FIRMS')  #FIRMS
        m.addLayer(alosTopographicDiversity, alosTopographicDiversityVis, 'ALOS TD') #ALOS TD


except ee.EEException as error:
    print(Fore.RED + '[ERROR. Map plotting or initialization failed] - {}'.format(str(error)[17:]))
    pass

In [ ]:
# CALLING MAP OBJECT

m

In [ ]:
# POST-PROCESSING FUNCTIONS

download_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
today = datetime.datetime.today()
time = today.strftime("%Y%m%d-%H.%M.%S")
timestr = str(time)

if save_map_html.clicked:
    if not os.path.exists(download_dir):
        os.makedirs(download_dir)
    html_file = os.path.join(download_dir, 'GAIA-{}.html'.format(timestr))
    m.to_html(filename=html_file, title='My Map', width='100%', height='880px')